In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

In [2]:
data = pd.read_csv('./Data/CS_dataset.csv').copy()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
del data['specialty_subgroup']

In [5]:
data

,id,online_booking,delay_in_days,new_patient,is_relative,booking_device,organization_type,patient_appt_number,previous_noshow,patient_age_group,appt_duration,visit_motive_name,month,day,start,created,received_reminder_sms,zipcode,no_show
0,1,0,14,no,NaN,not declared,Solo practitioner,2,0,55-64,30,Consultation & suivi,2,2,09:30:00,09:16:17.043422,1,06600,0
1,2,0,20,no,NaN,not declared,Solo practitioner,4,0,NaN,30,Consultation & suivi,3,2,15:45:00,12:39:01.706144,1,06110,0
2,3,1,6,no,False,mobile,Solo practitioner,13,0,35-44,45,Consultation & suivi,9,5,14:15:00,08:24:48.307585,1,92100,0
3,4,0,12,no,NaN,not declared,Practice,1,0,NaN,30,Extraction & chirurgie,12,3,17:30:00,16:41:04.374814,0,59224,0
4,5,0,57,no,False,not declared,Solo practitioner,2,0,55-64,90,Extraction & chirurgie,1,5,09:00:00,11:58:47.087595,1,69680,0
5,6,0,1,no,NaN,not declared,Practice,1,0,35-44,45,Consultation & suivi,6,4,10:45:00,11:53:04.434134,0,94160,0
6,7,0,8,no,NaN,desktop,Solo practitioner,5,0,65+,15,Pose & prothèse,4,2,14:45:00,15:45:27.936706,1,69006,0
7,8,0,60,yes,NaN,not declared,Salaried establishment,1,0,18-24,30,Première consultation,4,5,12:30:00,15:21:02.145633,1,76600,1
8,9,1,5,yes,True,mobile,Salaried establishment,1,0,35-44,80,Première consultation,1,2,15:00:00,02:04:30.493677,1,94800,0
9,10,0,0,no,NaN,not declared,Salaried establishment,3,0,35-44,15,Urgence,4,4,09:45:00,09:37:08.13074,0,91000,0


In [6]:
new_patient_class = list(data['organization_type'].value_counts().keys())
new_patient_class

['Practice',
 'Solo practitioner',
 'Salaried establishment',
 'Self-employed establishment',
 'Radiology']

## Let's encode the columns that are not numeric

In [7]:
new_patient_dummy = pd.get_dummies(data['new_patient'], prefix=['new_patient']).copy()

In [8]:
### The missing data isn't taken into account (NaN). Should we add it anyways ?
is_relative_dummy =  pd.get_dummies(data['is_relative'], prefix=['is_relative']).copy()

In [9]:
booking_device_dummy = pd.get_dummies(data['booking_device'], prefix=['booking_device']).copy()

In [10]:
organization_type_dummy = pd.get_dummies(data['organization_type'], prefix=['organization_type']).copy()

In [11]:
patient_age_group_dummy = pd.get_dummies(data['patient_age_group'], prefix=['patient_age_group']).copy()

In [12]:
visit_motive_name_dummy = pd.get_dummies(data['visit_motive_name'], prefix=['visit_motive_name']).copy()

In [13]:
month_dummy =  pd.get_dummies(data['month'], prefix=['month']).copy()

In [14]:
day_dummy = pd.get_dummies(data['day'], prefix=['day']).copy()

Now we drop these columns

In [15]:
data.drop(['new_patient', 'is_relative', 'booking_device', 'organization_type', 'patient_age_group', 'visit_motive_name', 'month', 'day'], axis=1, inplace = True)

And we concatenate them all

In [17]:
bucketized_df = pd.concat([data, patient_age_group_dummy, visit_motive_name_dummy,organization_type_dummy,booking_device_dummy, is_relative_dummy,new_patient_dummy, month_dummy, day_dummy], axis=1)


In [18]:
bucketized_df

,id,online_booking,delay_in_days,patient_appt_number,previous_noshow,appt_duration,start,created,received_reminder_sms,zipcode,...,['month']_10,['month']_11,['month']_12,['day']_0,['day']_1,['day']_2,['day']_3,['day']_4,['day']_5,['day']_6
0,1,0,14,2,0,30,09:30:00,09:16:17.043422,1,06600,...,0,0,0,0,0,1,0,0,0,0
1,2,0,20,4,0,30,15:45:00,12:39:01.706144,1,06110,...,0,0,0,0,0,1,0,0,0,0
2,3,1,6,13,0,45,14:15:00,08:24:48.307585,1,92100,...,0,0,0,0,0,0,0,0,1,0
3,4,0,12,1,0,30,17:30:00,16:41:04.374814,0,59224,...,0,0,1,0,0,0,1,0,0,0
4,5,0,57,2,0,90,09:00:00,11:58:47.087595,1,69680,...,0,0,0,0,0,0,0,0,1,0
5,6,0,1,1,0,45,10:45:00,11:53:04.434134,0,94160,...,0,0,0,0,0,0,0,1,0,0
6,7,0,8,5,0,15,14:45:00,15:45:27.936706,1,69006,...,0,0,0,0,0,1,0,0,0,0
7,8,0,60,1,0,30,12:30:00,15:21:02.145633,1,76600,...,0,0,0,0,0,0,0,0,1,0
8,9,1,5,1,0,80,15:00:00,02:04:30.493677,1,94800,...,0,0,0,0,0,1,0,0,0,0
9,10,0,0,3,0,15,09:45:00,09:37:08.13074,0,91000,...,0,0,0,0,0,0,0,1,0,0


And save them to a CSV file

In [19]:
bucketized_df.to_csv('encoded_data')

In [24]:
list(bucketized_df.columns)

['id',
 'online_booking',
 'delay_in_days',
 'patient_appt_number',
 'previous_noshow',
 'appt_duration',
 'start',
 'created',
 'received_reminder_sms',
 'zipcode',
 'no_show',
 "['patient_age_group']_0-17",
 "['patient_age_group']_18-24",
 "['patient_age_group']_25-34",
 "['patient_age_group']_35-44",
 "['patient_age_group']_45-54",
 "['patient_age_group']_55-64",
 "['patient_age_group']_65+",
 "['visit_motive_name']_Autre",
 "['visit_motive_name']_Bilan",
 "['visit_motive_name']_Consultation & suivi",
 "['visit_motive_name']_Extraction & chirurgie",
 "['visit_motive_name']_Orthodonthie",
 "['visit_motive_name']_Pose & prothèse",
 "['visit_motive_name']_Première consultation",
 "['visit_motive_name']_Soins légers",
 "['visit_motive_name']_Urgence",
 "['organization_type']_Practice",
 "['organization_type']_Radiology",
 "['organization_type']_Salaried establishment",
 "['organization_type']_Self-employed establishment",
 "['organization_type']_Solo practitioner",
 "['booking_device']_